In [2]:
!pip install python-dotenv
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.7 MB/s eta 0:00:00


In [15]:
import boto3
import os
from dotenv import load_dotenv
from google.colab import userdata

load_dotenv()

# Create an S3 client
s3 = boto3.client('s3')

my_session = boto3.session.Session(
    region_name=userdata.get('region'),
    aws_access_key_id=userdata.get('aws_access_key_id'),
    aws_secret_access_key=userdata.get('aws_secret_access_key'),
)

# Define your bucket and object key
bucket_name = userdata.get('bucket_name')
object_key = userdata.get('object_key')

try:
    # Get the object from S3 using the session's client
    s3_client = my_session.client('s3')
    response = s3_client.get_object(Bucket=bucket_name, Key=object_key)

    # Read and decode the content
    file_content = response['Body'].read().decode('utf-8')

    print("Response success")

except Exception as e:
    print(f"Error retrieving object: {e}")

Response success


In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

print(type(file_content))

<class 'str'>


In [20]:
import pandas as pd
import io

# Use io.StringIO to read the string content as a file
df = pd.read_csv(io.StringIO(file_content))

# Display the first 5 rows of the DataFrame
display(df.head())

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
